Generate remaining cycle

In [2]:
import pandas as pd
import numpy as np

In [3]:
columns=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]

read in data

In [11]:
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
test_results=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)

generate columns for RUL, cycle, rul failed, remaining cycle

In [12]:
test_results.columns=["rul", "null"]

test_results['id']=test_results.index+1
test_results.drop(["null"],axis=1,inplace=True)
rul = pd.DataFrame(test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
test_results['rul_failed']=test_results['rul']+rul['max']
test_results.drop(["rul"],axis=1,inplace=True)
test=test.merge(test_results,on=['id'],how='left')
test["remaining_cycle"]=test["rul_failed"]-test["cycle"]



drop null values and rul failed for test set

In [14]:
df_train=train.drop(["sensor22","sensor23"],axis=1)
df_test=test.drop(["sensor22","sensor23"],axis=1)
df_test.drop(["rul_failed"],axis=1,inplace=True)
df_train['remaining_cycle'] = df_train.groupby(['id'])['cycle'].transform(max)-df_train['cycle']

display columns

In [15]:
test.head()


,id,cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,sensor22,sensor23,rul_failed,remaining_cycle
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,0.03,392,2388,100.0,38.86,23.3735,NaN,NaN,143,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,0.03,393,2388,100.0,39.02,23.3916,NaN,NaN,143,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,0.03,393,2388,100.0,39.08,23.4166,NaN,NaN,143,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,0.03,391,2388,100.0,39.00,23.3737,NaN,NaN,143,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,0.03,390,2388,100.0,38.99,23.4130,NaN,NaN,143,138


generate label

In [16]:
# set CYCLES variable to predict if the engine will fail within the next n cycles
CYCLES = 20 # predict if the engine will fail in the next 20 cycles
df_train['label'] = df_train['remaining_cycle'].apply(lambda x: 1 if x <= CYCLES else 0)
df_test['label'] = df_test['remaining_cycle'].apply(lambda x: 1 if x <= CYCLES else 0)

In [25]:
df_train["sensor17"] = pd.to_numeric(df_train["sensor17"], downcast="float")
df_test["sensor17"] = pd.to_numeric(df_test["sensor17"], downcast="float")
df_train["sensor18"] = pd.to_numeric(df_train["sensor18"], downcast="float")
df_test["sensor18"] = pd.to_numeric(df_test["sensor18"], downcast="float")
df_train["sensor19"] = pd.to_numeric(df_train["sensor19"], downcast="float")
df_test["sensor19"] = pd.to_numeric(df_test["sensor19"], downcast="float")


In [23]:
df_train.dtypes

id                   int64
cycle                int64
op1                float64
op2                float64
op3                float64
sensor1            float64
sensor2            float64
sensor3            float64
sensor4            float64
sensor5            float64
sensor6            float64
sensor7            float64
sensor8            float64
sensor9            float64
sensor10           float64
sensor11           float64
sensor12           float64
sensor13           float64
sensor14           float64
sensor15           float64
sensor16           float64
sensor17           float32
sensor18           float32
sensor19           float32
sensor20           float64
sensor21           float64
remaining_cycle      int64
label                int64
dtype: object

save the file

In [26]:
#parquet
df_train.to_parquet('engine-train.parquet')
df_test.to_parquet('engine-test.parquet')

#csv
df_train.to_csv('engine-train.csv')
df_test.to_csv('engine-test.csv')